In [2]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [ ]:
#TGAC
file_deg = '../data/DEGs_by_RNAseq_21dpi_wheat.txt'
file_annotation = '/home/juan/Desktop/juan/bio/data/genomes/TGAC/Triticum_aestivum.TGACv1.39.clean.gff3'
dir_fasta = "/home/juan/Desktop/juan/bio/data/genomes/TGAC/by_scaff/"
file_out_fasta = "/home/juan/Desktop/juan/bio/mrcv/data/TGAC_DEG_SEQS.fasta"
gene_id = 'ID=gene:'
df_list = pd.read_csv(file_deg, index_col=False, sep='\t', comment='#')
genes = df_list.Gene.tolist() 
print(len(genes))

In [3]:
#IWGSC
file_deg = '/home/juan/Desktop/juan/bio/mrcv/data/res/rna_deg/DEG-normalized.csv'
file_annotation = '/home/juan/Desktop/juan/bio/mrcv/data/data/iwgsc/Triticum_aestivum.IWGSC.41.gff3'
dir_fasta = "/home/juan/Desktop/juan/bio/mrcv/data/data/iwgsc/by_chr/"
file_out_fasta = "/home/juan/Desktop/juan/bio/mrcv/data/res/DEG_SEQS.fasta"
gene_id = 'ID=gene:'
df_list = pd.read_csv(file_deg, index_col=False, sep=',', comment='#')
genes = df_list.gene.tolist() 
print(len(genes))

3019


In [6]:
len(df_list.index)

3019

In [7]:
df_list.head()

,Unnamed: 0,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,C1.counts.txt,C2.counts.txt,C3.counts.txt,C4.counts.txt,T1.counts.txt,T2.counts.txt,T3.counts.txt,T4.counts.txt
0,1,gene:TraesCS2B02G286300,320.013057,2.445059,0.225052,10.864392,1.703534e-27,5.369907e-23,582.257421,553.695714,476.353565,549.686360,77.245985,81.333777,93.617622,145.914014
1,2,gene:TraesCS2B02G336800,2294.525203,-2.909846,0.268059,-10.855254,1.882824e-27,5.369907e-23,532.840739,512.938938,500.171243,609.940442,4976.988116,5270.112921,2148.061490,3805.147731
2,3,gene:TraesCS6D02G241200,191.594697,2.320181,0.223227,10.393825,2.645258e-25,5.029605e-21,300.797192,294.244042,391.999288,289.642428,71.379201,56.854679,81.272441,46.568302
3,4,gene:TraesCS7B02G073900,681.317343,2.012952,0.195772,10.282137,8.482575e-25,1.209636e-20,1254.753999,895.655005,1081.719554,1136.370841,294.316979,273.218320,318.917175,195.586870
4,5,gene:TraesCS7B02G093100,114.688241,2.931565,0.287328,10.202859,1.925209e-24,2.114932e-20,249.231959,166.009307,137.944053,257.929754,25.422729,25.268746,30.862952,24.836428


In [8]:
len(df_list[df_list.log2FoldChange > 0].index), len(df_list[df_list.log2FoldChange < 0].index), 

(1272, 1747)

In [4]:
df_gff = pd.read_csv(file_annotation, index_col=False, sep='\t', header=None, comment='#')
df_gff.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
df_gff = df_gff[df_gff.feature == 'gene']
print(len(df_gff.index))
df_gff.head(3)

107891


,seqname,source,feature,start,end,score,strand,frame,attribute
34,1A,IWGSC,gene,40098,70338,.,-,.,ID=gene:TraesCS1A02G000100;biotype=protein_cod...
65,1A,IWGSC,gene,70239,89245,.,+,.,ID=gene:TraesCS1A02G000200;biotype=protein_cod...
90,1A,IWGSC,gene,95906,104903,.,+,.,ID=gene:TraesCS1A02G000300;biotype=protein_cod...


In [ ]:
buffer_seqs = []
padding = 2000
for gene in genes:
    gene = gene.replace("gene:","")
    df_extract = df_gff[(df_gff.attribute.str.contains(gene_id + gene))]
    gene_gff = df_extract.iloc[0]
    end = max(gene_gff.end, gene_gff.start)
    start = min(gene_gff.end, gene_gff.start)
    start = max(start-padding, 0)
    end += padding
    record = SeqIO.read(dir_fasta + gene_gff.seqname + '.fasta', 'fasta')
    clean_seq = ''.join(str(record.seq).splitlines())
    new_seq = clean_seq[start:end]
    desc = "seq:" + str(gene_gff.seqname)
    desc += " start:" + str(start)
    desc += " end:" + str(end)
    desc += " strand:" + str(gene_gff.strand)
    seq = SeqRecord(Seq(new_seq), id=gene, description=desc)
    buffer_seqs.append(seq)


In [13]:
SeqIO.write(buffer_seqs, file_out_fasta, "fasta")

1143